# Playground2: Kilosort + Template Metrics

In [26]:
from pathlib import Path
import spikeinterface.full as si

print(f"SpikeInterface version: {si.__version__}")

SpikeInterface version: 0.103.3


In [49]:
# Check if Kilosort output already exists

# For kilosort/phy output files we can use the read_phy
# most formats will have a read_xx that can used.
analyzer = si.load_sorting_analyzer('/Users/jf5479/Downloads/kilosort4_sa/')

# if kilosort_output.exists() and (kilosort_output / "spike_times.npy").exists():
#     print(f"Kilosort output already exists at: {kilosort_output}")
#     print("Loading existing sorting results...")
#     sorting = si.read_sorter_folder(kilosort_output)
# else:
#     print(f"Running Kilosort4, output will be saved to: {kilosort_output}")
#     print(f"Installed sorters: {si.installed_sorters()}")

#     # Run Kilosort4
#     sorting = si.run_sorter(
#         sorter_name="kilosort4",
#         recording=rec_preprocessed,
#         folder=kilosort_output,
#         verbose=True,
#         remove_existing_folder=True,  # Remove any failed previous attempts
#     )
#     print("Kilosort4 completed!")

print(f"Sorting result: {sorting}")
print(f"Number of units: {len(sorting.unit_ids)}")

/Users/jf5479/Dropbox/Python/spikeinterface/src/spikeinterface/core/base.py:1117: UserWarning: Versions are not the same. This might lead to compatibility errors. Using spikeinterface==0.101.2 is recommended
  warnings.warn(


Sorting result: SortingAnalyzer: 384 channels - 330 units - 1 segments - binary_folder - sparse
Loaded 11 extensions: template_metrics, random_spikes, unit_locations, quality_metrics, waveforms, spike_amplitudes, templates, spike_locations, correlograms, template_similarity, noise_levels
Number of units: 330


In [50]:
# Random spikes selection
if not analyzer.has_extension("random_spikes"):
    print("Computing random_spikes...")
    analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)

In [51]:
# Waveforms
if not analyzer.has_extension("waveforms"):
    print("Computing waveforms...")
    analyzer.compute("waveforms", ms_before=1.5, ms_after=2.0, **job_kwargs)

In [52]:
# Templates
if not analyzer.has_extension("templates"):
    print("Computing templates...")
    analyzer.compute("templates", operators=["average", "median", "std"])

In [53]:
# Noise levels
if not analyzer.has_extension("noise_levels"):
    print("Computing noise_levels...")
    analyzer.compute("noise_levels")

##  Compute Template Metrics

In [58]:
# Compute template metrics with multi-channel metrics included
analyzer.compute(
    "template_metrics",
    smooth=True,              # Enable/disable smoothing
    smooth_method='svd',
    svd_n_components=3,
    smooth_window_frac=0.1,   # Window as fraction of template length
    smooth_polyorder=3,       # Polynomial order
    min_thresh_detect_peaks_troughs=0.4
)


TypeError: ComputeTemplateMetrics._set_params() got an unexpected keyword argument 'smooth_method'

In [ ]:


# Get the metrics as a DataFrame
template_metrics = analyzer.get_extension("template_metrics").get_data()
template_metrics

,peak_to_valley,peak_trough_ratio,half_width,repolarization_slope,recovery_slope,num_positive_peaks,num_negative_peaks
0,0.001173,-0.440069,0.001740,16271.944574,-12828.347767,1,2
1,0.000657,-0.200979,0.000163,321619.078371,-10908.856693,1,1
2,0.001320,-0.374601,0.000647,12679.478171,-88050.930624,1,2
3,0.000653,-0.242145,0.000220,119110.411447,-7646.450397,1,1
4,0.000820,-0.275446,0.000260,75484.461584,-7431.037919,2,1
...,...,...,...,...,...,...,...
325,0.001383,-0.250669,0.000517,20844.603913,-7716.307155,1,1
326,0.000603,-0.686821,0.000397,72116.897018,-21101.644706,1,2
327,0.001190,-0.264008,0.000543,37373.373888,-7624.695940,2,1
328,0.001090,-0.327231,0.000530,28242.461115,-9136.500514,2,1


In [ ]:
# Spike amplitudes
if not analyzer.has_extension("spike_amplitudes"):
    print("Computing spike_amplitudes...")
    analyzer.compute("spike_amplitudes", **job_kwargs)

In [ ]:
# Correlograms
if not analyzer.has_extension("correlograms"):
    print("Computing correlograms...")
    analyzer.compute("correlograms")

In [ ]:
# Quality metrics
if not analyzer.has_extension("quality_metrics"):
    print("Computing quality_metrics...")
    analyzer.compute("quality_metrics")

quality_metrics = analyzer.get_extension("quality_metrics").get_data()
quality_metrics

## Summary

In [ ]:
print(f"Total units: {len(sorting.unit_ids)}")
print(f"Analyzer saved to: {analyzer_folder}")
print(f"\nAvailable extensions:")
for ext_name in analyzer.get_loaded_extension_names():
    print(f"  - {ext_name}")

In [ ]:
# Combine metrics
combined_metrics = template_metrics.join(quality_metrics, how="outer")
combined_metrics

In [ ]:
# Save metrics to CSV
output_folder.mkdir(parents=True, exist_ok=True)
metrics_csv = output_folder / "combined_metrics.csv"
combined_metrics.to_csv(metrics_csv)
print(f"Metrics saved to: {metrics_csv}")